<img src = "https://images2.imgbox.com/a5/72/7ZbDUHlf_o.jpg" width="200">

## Exercício
---

**Entrega individual de um notebook com esses passos indentificados.**

Com o dataset do projeto, faça o seguinte:
    
1. Crie uma pipeline específica para as seguintes variáveis e realize os tratamentos necessários para transformar essa informação em número ordenado:
    - `NAME_EDUCATION_TYPE`  
    
2. Crie uma pipeline específica para as seguintes variáveis, realizando um One Hot Encoder (leiam a documentação e setem o parâmetro drop conforme fizer sentido para vocês):
    - `FLAG_OWN_CAR`
    - `FLAG_OWN_REALTY`
    - `NAME_INCOME_TYPE`
    - `NAME_FAMILY_STATUS`
    - `OCCUPATION_TYPE`  

3. Crie uma pipeline específica para todas as outras variáveis que não são do tipo object. Adicione na pipeline a imputação de missing com a mediana. Com base nos próximos passos, responda a seguinte pergunta:
    - *É necessário realizar o standard scaler nessas variáveis?*  
    
4. Crie uma outra pipeline, que unirá todas as pipelines anteriores. Além de unir os steps passados, essa pipeline deve aplicar uma seleção de features baseada em um modelo de regressão logística. Resposta a seguinte pergunta:
    - *Qual penalidade deve ser usada? Por que?*  
    
5. Após a etapa de seleção de features da pipeline do passo anterior, adicione um modelo de árvore de decisão com `max_depth = 5` e treine o modelo, avaliando as métricas.  

6. Faça uma busca em grid nos hiperparâmetros `min_samples_leaf` e `max_depth`, retreine o modelo e avalie as métricas. Responda a seguinte pergunta:
    - *Houve ganho em performance? Se sim, de quanto?*   
    
7. Substitua o modelo de árvore de decisão por um modelo XGBoost, AdaBoost ou LightGBM, realizando também uma busca nos hiperparâmetros (lembre-se de alterar o grid de hiperparâmetros a ser testado). Avalie:

- *Como ficou a performance do modelo usando o novo algoritmo?*
- *A variável `pipe_cat__OCCUPATION_TYPE_nan` está bonificando ou penalizando o cliente? Ou seja, se o cliente tiver ocupação como missing, ele tem tendência a ser melhor ou pior pagador?*

In [1]:
# importando bibliotecas
import numpy as np
import pandas as pd


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
# importando dados
df = pd.read_csv('dados/application_train.csv')

In [3]:
df = df.sample(frac=0.1)

In [4]:
# o dataset possui 122 variáveis e 24601 observações
df.shape

(24601, 122)

In [5]:
# cinco primeiras observações do dataframe
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
22546,380404,0,Cash loans,F,Y,Y,0,292500.0,1375380.0,49531.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
18236,216051,0,Revolving loans,F,Y,Y,1,90000.0,180000.0,9000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
155103,342201,1,Cash loans,M,N,Y,0,112500.0,328500.0,19980.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4778,419325,0,Cash loans,M,Y,N,0,180000.0,1009566.0,30609.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
105773,392857,0,Cash loans,F,N,Y,0,90000.0,970380.0,28503.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# cinco últimas observações do dataframe
df.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
91532,369598,0,Cash loans,M,N,Y,1,180000.0,485640.0,34668.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
46651,390822,0,Cash loans,F,N,Y,0,157500.0,573628.5,22878.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,4.0
138695,382041,0,Cash loans,F,Y,Y,0,67500.0,1256400.0,36864.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
201252,382211,0,Cash loans,F,N,Y,0,90000.0,284256.0,29979.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
227882,424224,0,Cash loans,F,N,Y,0,202500.0,1980864.0,68985.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# separando dados
X = df.drop(['TARGET','SK_ID_CURR'], axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

1. Crie uma pipeline específica para as seguintes variáveis e realize os tratamentos necessários para transformar essa informação em número ordenado:
    - `NAME_EDUCATION_TYPE`

In [8]:
# criando pipeline com ordinal encoder
lista_ordenada = ['Lower secondary','Secondary / secondary special','Incomplete higher','Higher education','Academic degree']
var_edu = ['NAME_EDUCATION_TYPE']

pipe_edu = Pipeline(steps = [
    ('input', SimpleImputer(missing_values='', strategy='most_frequent')),
    ('trata_edu', OrdinalEncoder(categories = [lista_ordenada]))
])

2. Crie uma pipeline específica para as seguintes variáveis, realizando um One Hot Encoder (leiam a documentação e setem o parâmetro drop conforme fizer sentido para vocês):
    - `FLAG_OWN_CAR`
    - `FLAG_OWN_REALTY`
    - `NAME_INCOME_TYPE`
    - `NAME_FAMILY_STATUS`
    - `OCCUPATION_TYPE`

In [9]:
# criando pipeline aplicando one hot encoding 
var_cat = ['FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','OCCUPATION_TYPE']

pipe_cat = Pipeline(steps = [
    ('trata_cat', OneHotEncoder(sparse = False, drop = 'if_binary', handle_unknown='ignore'))
])

3. Crie uma pipeline específica para todas as outras variáveis que não são do tipo object. Adicione na pipeline a imputação de missing com a mediana. Com base nos próximos passos, responda a seguinte pergunta:
    - *É necessário realizar o standard scaler nessas variáveis?* R.: Sim, já que será utilizada a regressão, a normalização irá influenciar nos resultados.

In [10]:
# selecionando variáveis numéricas
var_num = df.select_dtypes(exclude = ['object']).drop(['TARGET', 'SK_ID_CURR'], axis = 1).columns.tolist()

# criando pipeline
pipe_num = Pipeline(steps = [
    ('trata_num', SimpleImputer(missing_values = np.nan, strategy = 'median')),
    ('scaling', StandardScaler())
])

4. Crie uma outra pipeline, que unirá todas as pipelines anteriores. Além de unir os steps passados, essa pipeline deve aplicar uma seleção de features baseada em um modelo de regressão logística. Resposta a seguinte pergunta:
    - *Qual penalidade deve ser usada? Por que?* R.: L1, porque também é possível selecionar variáveis relevantes ao modelo.

In [11]:
# unindo pipelines
pipe_preproc = ColumnTransformer(transformers = [
    ('pipe_edu', pipe_edu, var_edu),
    ('pipe_cat', pipe_cat, var_cat),
    ('pipe_num', pipe_num, var_num),
])

# definindo o modelo de regressão logística
modelo_logreg = LogisticRegressionCV(
    penalty = 'l1', 
    solver = 'liblinear', 
    Cs = np.arange(0.01, 0.1, 0.01), 
    scoring = 'roc_auc'
)

# pipeline para seleção de features
pipe_features = Pipeline(steps = [
    ('pipe_features', SelectFromModel(estimator = modelo_logreg, max_features=10, threshold=0.1))
])

# criando a pipeline final
pipe_final = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features)
])

In [12]:
pipe_final.fit_transform(df, df['TARGET'])

array([[ 3.        ,  0.        ,  0.        , ...,  1.33100026,
         0.28665055,  0.63517377],
       [ 1.        ,  0.        ,  0.        , ..., -0.19721209,
        -1.62383795, -1.57437232],
       [ 1.        ,  0.        ,  0.        , ..., -1.40336296,
         0.1234947 ,  0.63517377],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.50981824,
        -0.90431362,  0.63517377],
       [ 1.        ,  0.        ,  0.        , ...,  0.97621441,
         0.1234947 ,  0.63517377],
       [ 1.        ,  0.        ,  0.        , ...,  1.12719991,
         0.20531441,  0.63517377]])

5. Após a etapa de seleção de features da pipeline do passo anterior, adicione um modelo de árvore de decisão com `max_depth = 5` e treine o modelo, avaliando as métricas.

In [13]:
# definindo o modelo de árvore de decisão
modelo_dt = DecisionTreeClassifier(max_depth = 6, random_state = 42)

# criando a pipeline final
pipe_final = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features),
    ('modelo', modelo_dt)
])

In [14]:
pipe_final.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('input',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                 

In [15]:
print('ROC-AUC treino:', roc_auc_score(y_train, (pipe_final.predict_proba(X_train)[:,1])))
print('ROC-AUC teste:', roc_auc_score(y_test, (pipe_final.predict_proba(X_test)[:,1])))

ROC-AUC treino: 0.7311827253025104
ROC-AUC teste: 0.6744107350452431


6. Faça uma busca em grid nos hiperparâmetros `min_samples_leaf` e `max_depth`, retreine o modelo e avalie as métricas. Responda a seguinte pergunta:
    - *Houve ganho em performance? Se sim, de quanto?* R.: Ganho de performance irrelevante.

In [16]:
parametros = {
    'modelo__max_depth': [20, 50, 100],
    'modelo__min_samples_leaf': [2, 3, 5]
}

gscv = GridSearchCV(
    estimator = pipe_final,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

In [17]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('pipe_edu',
                                                                         Pipeline(steps=[('input',
                                                                                          SimpleImputer(missing_values='',
                                                                                                        strategy='most_frequent')),
                                                                                         ('trata_edu',
                                                                                          OrdinalEncoder(categories=[['Lower '
                                                                                                                      'secondary',
                                                                                                                      'Secondary '

In [18]:
print('ROC-AUC treino:', roc_auc_score(y_train, (gscv.predict_proba(X_train)[:,1])))
print('ROC-AUC teste:', roc_auc_score(y_test, (gscv.predict_proba(X_test )[:,1])))

ROC-AUC treino: 0.9500747697309011
ROC-AUC teste: 0.5702148394715312


In [19]:
gscv.best_params_

{'modelo__max_depth': 20, 'modelo__min_samples_leaf': 5}

In [20]:
gscv.best_estimator_

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('input',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                 

7. Substitua o modelo de árvore de decisão por um modelo XGBoost, AdaBoost ou LightGBM, realizando também uma busca nos hiperparâmetros (lembre-se de alterar o grid de hiperparâmetros a ser testado). Avalie:

- *Como ficou a performance do modelo usando o novo algoritmo?*
- *A variável `pipe_cat__OCCUPATION_TYPE_nan` está bonificando ou penalizando o cliente? Ou seja, se o cliente tiver ocupação como missing, ele tem tendência a ser melhor ou pior pagador?*

In [28]:
!pip install -q lightgbm

In [29]:
from lightgbm import LGBMClassifier

# definindo o modelo 
modelo_lgbm = LGBMClassifier(n_estimators=3)

parametros = {
    'modelo__n_estimators': [100, 300, 500],
}

# criando a pipeline final
pipe_final2 = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features),
    ('modelo', modelo_lgbm)
])

gscv = GridSearchCV(
    estimator = pipe_final2,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

In [30]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('pipe_edu',
                                                                         Pipeline(steps=[('input',
                                                                                          SimpleImputer(missing_values='',
                                                                                                        strategy='most_frequent')),
                                                                                         ('trata_edu',
                                                                                          OrdinalEncoder(categories=[['Lower '
                                                                                                                      'secondary',
                                                                                                                      'Secondary '

In [32]:
gscv.best_params_

{'modelo__n_estimators': 100}

In [33]:
gscv.best_estimator_

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('input',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                 

In [31]:
print('ROC-AUC treino:', roc_auc_score(y_train, (gscv.predict_proba(X_train)[:,1])))
print('ROC-AUC teste:', roc_auc_score(y_test, (gscv.predict_proba(X_test )[:,1])))

ROC-AUC treino: 0.911973248148817
ROC-AUC teste: 0.7064466360393571


---